# Flow computation

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import subprocess
import sys
def get_repo_root():
    """Get the root directory of the repo."""
    dir_in_repo = os.path.dirname(os.path.abspath('__file__')) # os.getcwd()
    return subprocess.check_output('git rev-parse --show-toplevel'.split(),
                                   cwd=dir_in_repo,
                                   universal_newlines=True).rstrip()
sys.path.append(get_repo_root())
ROOT_dir = get_repo_root()

In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np

osm_folder = ROOT_dir + '/dbs/osm/'
city = 'dublin'
network = osm_folder + f'{city}.shp'

## 1 Load data

In [3]:
gdf = gpd.read_file(network)
gdf['osm_id'] = gdf['osm_id'].astype(int)
df = pd.read_csv(ROOT_dir + '/dbs/flow2.csv')
df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'], inplace=True)

In [4]:
df_n = pd.merge(df, gdf.loc[:, ['osm_id', 'oneway', 'lanes']].drop_duplicates(subset=['osm_id']),
                left_on='OSM_ID', right_on='osm_id', how='left')
df_n.loc[:, 'lanes'] = df_n.loc[:, 'lanes'].apply(lambda x: int(x) if x in ['4', '3', '2', '1'] else 1)
df_n.loc[:, 'oneway'] = df_n.loc[:, 'oneway'].astype(int)
df_n.head()

,Time,Here_segmentID,Here_Speed_uncap,Here_FFS,Flow,Flow_Sensor_Number,OSM_ID,OSM_Max_Speed,osm_id,oneway,lanes
0,2018-02-01 00:00:00,2_18211-,25.595833,34.0,1570,37,4408238,50,4408238,1,3
1,2018-02-01 00:00:00,2_5202-,24.006667,35.0,1570,37,4408238,50,4408238,1,3
2,2018-02-01 00:00:00,2_4224-,28.519167,33.0,1570,37,361409608,50,361409608,1,2
3,2018-02-01 00:00:00,2_18210-,30.400833,33.0,1570,37,3789252,50,3789252,0,3
4,2018-02-01 00:00:00,2_18210-,30.400833,33.0,1570,37,3789252,50,3789252,0,3


In [5]:
df_n.groupby(['oneway', 'lanes']).size()

oneway  lanes
0       1         21626
        2          9830
        3        106152
        4          5889
1       1          1966
        2          5898
        3         45218
dtype: int64

In [9]:
print('Number of road segments:', len(df_n.Here_segmentID.unique()))

Number of road segments: 48


## 2 Define capacity theoretically

In [6]:
capacity_dict = {(1, 1): 1200, (1, 0): 700, (2, 1): 2400, (2, 0): 1500,
                 (3, 1): 3600, (3, 0): 2800, (4, 1): 4800, (4, 0): 4000}

In [8]:
df_n.loc[:, 'capacity'] = df_n.apply(lambda row: capacity_dict[(row['lanes'], row['oneway'])], axis=1)
df_n.to_csv(ROOT_dir + '/dbs/flow2m.csv', index=False)

## 3 Test the simple form of BPR

In [9]:
df_n = pd.read_csv(ROOT_dir + '/dbs/flow2m.csv')
def capacity_estimation(a=None, L=None, u=None, RT=None, l=None):
    return 1000*l/(0.278 * RT + u/(26 * a) + L)
df_n.loc[:, 'capacity_est'] = df_n.apply(lambda row: capacity_estimation(a=2.5, L=4.5, u=row['OSM_Max_Speed'],
                                                                         RT=1.1, l=row['lanes']), axis=1)

In [12]:
alpha = 0.15
beta = 4
# Here_FFS
df_n.loc[:, 'flow'] = df_n.capacity * ( (df_n.Here_FFS / df_n.Here_Speed_uncap - 1) / alpha )**(1/beta)